<a href="https://colab.research.google.com/github/sakuna47/AI_Maze/blob/Code/AI_MAZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random  # For random selection of start, goal, and barriers
import math  # For calculating diagonal costs (sqrt(2))
import heapq  # For priority queue in UCS and A* search
import statistics  # For calculating mean and variance in analysis
from collections import deque  # For BFS in solvability check

In [2]:
# Convert a node number (0-35) to (x, y) coordinates in a 6x6 grid
# x = node % 6 (column), y = node // 6 (row)
# Example: Node 15 -> (x=2, y=3)
def get_coordinates(node):
    return (node % 6, node // 6)


In [3]:
# Get valid neighbors of a node with their edge costs
# Neighbors are in 8 directions: horizontal, vertical, diagonal
# Barriers block movement; neighbors are sorted by node number for deterministic exploration
def get_neighbors(node, barriers):
    x, y = get_coordinates(node)  # Get current node's coordinates
    # Define 8 possible directions: up, down, left, right, and diagonals
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    neighbors = []
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy  # Calculate neighbor's coordinates
        # Check if the neighbor is within the 6x6 grid
        if 0 <= new_x < 6 and 0 <= new_y < 6:
            neighbor = new_y * 6 + new_x  # Convert coordinates back to node number
            if neighbor not in barriers:  # Exclude barriers
                # Cost is 1 for horizontal/vertical moves, sqrt(2) for diagonal moves
                cost = 1 if dx == 0 or dy == 0 else math.sqrt(2)
                neighbors.append((neighbor, cost))
    # Sort neighbors by node number to ensure consistent exploration order
    # Example: For node 8, neighbors should be processed as 2, 7, 9, 14
    neighbors.sort(key=lambda x: x[0])
    return neighbors